# How much average monthly income I could expect from my investments in terms of lease/rent after upgrading properties, paying mortgage, and paying taxes. Calculate my ~ROI.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### Assumptions

- Apply an average upgrade cost to all properties (2%-6%, assume 4%) [Google](https://www.google.com/search?sca_esv=f9439907e5543505&rlz=1C5CHFA_enUS1066US1066&sxsrf=ADLYWIKGALe9lErWdMYpzzi_ACDXvpSxuQ:1733020033405&q=So+if+I+buy+a+house,+what+is+the+average+cost+to+spruce+up+the+house+(i.+paint,+carpet,+etc)?+Is+there+a+percentage+I+can+apply+to+the+home+value+to+get+this+value?&spell=1&sa=X&ved=2ahUKEwjY-OrbwoWKAxVarYkEHXoMMNsQBSgAegQIDRAB&biw=2560&bih=1318&dpr=1)
- Mortage rate: 6.31% **Source:** [Bankrate](https://www.bankrate.com/mortgages/mortgage-rates/)
- 30 year fixed mortgage
- Property tax rates by state imported as property_tax_rates.csv **Source:** [Maptive](https://www.maptive.com/interactive-maps/property-taxes-by-state/)
- Renters pay all utilities
- Rent rates: Imported as rent_rates.csv **Source:** [Rentdata](https://www.rentdata.org/states/2024)